In [1]:
import os
import cv2
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

In [2]:
path_benchmark = "D:\\Projects\\Datasets\\Exams\\benchmark_final.avi"
path_label = "D:\\Projects\\Datasets\\Exams\\benchmark_final\\label\\label_benchmark_final.csv"
path_result_excuse = "D:\\Projects\\Datasets\\Exams\\benchmark_final\\results\\result.txt"
path_result_deepRN = "D:\\Projects\\Results\\Qualificacao\\DeepRN\\label.csv"

In [3]:
file_label = pd.read_csv(path_label)
file_label

,frame,center_x,center_y
0,0,272,222
1,1,271,223
2,2,269,226
3,3,269,228
4,4,269,229
...,...,...,...
4395,4395,191,162
4396,4396,195,162
4397,4397,201,158
4398,4398,217,143


In [4]:
file_excuse = pd.read_csv(path_result_excuse)
file_excuse

,img,position_x,position_y
0,1.png,238.000000,285.000000
1,10.png,222.000000,295.000000
2,100.png,191.000000,308.000000
3,1000.png,78.144545,419.346104
4,1001.png,280.000000,157.000000
...,...,...,...
4395,995.png,201.000000,280.000000
4396,996.png,0.000000,0.000000
4397,997.png,171.000000,299.000000
4398,998.png,287.100910,481.816606


In [5]:
file_deepRN = pd.read_csv(path_result_deepRN)
file_deepRN = file_deepRN.drop(['patient', 'param', 'radius', 'flash_algorithm', 'flash_information',
                                'color_flash', 'eye_size', 'img_mean', 'img_std', 'img_median'], axis=1)
file_deepRN

,frame,center_x,center_y
0,0,272.038625,222.280528
1,1,271.061452,223.986410
2,2,269.902751,226.316935
3,3,269.690399,228.427734
4,4,269.576441,229.833323
...,...,...,...
4395,4395,191.610158,162.026545
4396,4396,195.289307,162.813568
4397,4397,201.530584,158.175456
4398,4398,217.249247,143.932317


In [6]:
def calcule_points_position(center):
    size = 5
    p1 = (center[0], center[1] - size), (center[0], center[1] + size)
    p2 = (center[0] - size, center[1]), (center[0] + size, center[1])
    return p1, p2

In [8]:
def validate_in_area(position_center, position_point_validate, radius):
    square = ((position_point_validate[0] - position_center[0]) ** 2) + ((position_point_validate[1] - position_center[1]) ** 2)
    return math.sqrt(square) < radius

In [31]:
benchmark = cv2.VideoCapture(path_benchmark)

fourcc = cv2.VideoWriter_fourcc(*'MP42')
video = cv2.VideoWriter('compare_excuse_deepRN.avi', fourcc, float(17), (1280, 480))

ret, frame = benchmark.read()
number_frame = 0

on_5_excuse, on_10_excuse = 0, 0
on_5_deepRN, on_10_deepRN = 0, 0
while ret:
    filter_label = file_label['frame'] == number_frame
    data_label = file_label[filter_label]
    center_label = (int(data_label['center_x'].values[0]), int(data_label['center_y'].values[0]))
    
    filter_excuse = file_excuse['img'] == '{}.png'.format(number_frame + 1)
    data_excuse = file_excuse[filter_excuse]    
    center_excuse = (int(data_excuse['position_y'].values[0]), int(data_excuse['position_x'].values[0]))
    
    filter_deepRN = file_deepRN['frame'] == number_frame
    data_deepRN = file_deepRN[filter_deepRN]
    center_deepRN = (int(data_deepRN['center_x'].values[0]), int(data_deepRN['center_y'].values[0]))
    
    proportion_image_y_5 = int(frame.shape[0] * 0.05)
    proportion_image_y_10 = int(frame.shape[0] * 0.1)
    proportion_image_y_15 = int(frame.shape[0] * 0.15)
    
    # frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2RGB)
    img_excuse = np.copy(frame)
    img_deepRN = np.copy(frame)
    
    color_circle = (255, 255, 0)
    color_position = (255, 255, 255)
    color_text = (255, 255, 255)    
    
    cv2.circle(img_excuse, center_label, proportion_image_y_5, color_circle, 1)
    # cv2.circle(img_excuse, center_label, proportion_image_y_10, color_circle, 1)
    # cv2.circle(img_excuse, center_label, proportion_image_y_15, color_circle, 1)
    
    cv2.circle(img_deepRN, center_label, proportion_image_y_5, color_circle, 1)
    # cv2.circle(img_deepRN, center_label, proportion_image_y_10, color_circle, 1)
    # cv2.circle(img_deepRN, center_label, proportion_image_y_15, color_circle, 1)    
    
    if (center_excuse[0] > 0) and (center_excuse[1] > 0):
        p1, p2 = calcule_points_position(center_excuse)
        cv2.line(img_excuse, p1[0], p1[1], color_position, 2)
        cv2.line(img_excuse, p2[0], p2[1], color_position, 2)
        
        color_text_excuse = (0, 0, 255)
        if validate_in_area(center_label, center_excuse, proportion_image_y_5):            
            on_5_excuse += 1
            color_text_excuse = color_text
    
    if (center_deepRN[0] > 0) and (center_deepRN[1] > 0):
        p1, p2 = calcule_points_position(center_deepRN)
        cv2.line(img_deepRN, p1[0], p1[1], color_position, 2)
        cv2.line(img_deepRN, p2[0], p2[1], color_position, 2)
        
        color_text_deepRN = (0, 0, 255)
        if validate_in_area(center_label, center_deepRN, proportion_image_y_5):            
            on_5_deepRN += 1 
            color_text_deepRN = color_text
    
    cv2.putText(img_excuse, 'Excuse', (70, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, color_text, 1, cv2.LINE_AA)
    cv2.putText(img_excuse, 'Accuracy: %.4f' % (on_5_excuse / (number_frame + 1)), (430, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color_text, 1, cv2.LINE_AA)
    cv2.putText(img_excuse, 'Label = {}'.format(center_label), (430, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color_text, 1, cv2.LINE_AA)
    cv2.putText(img_excuse, 'Estimate = {}'.format(center_excuse), (430, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color_text_excuse, 1, cv2.LINE_AA)
        
    cv2.putText(img_deepRN, 'DeepRN', (70, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, color_text, 1, cv2.LINE_AA)
    cv2.putText(img_deepRN, 'Accuracy: %.4f' % (on_5_deepRN / (number_frame + 1)), (430, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color_text, 1, cv2.LINE_AA)
    cv2.putText(img_deepRN, 'Label = {}'.format(center_label), (430, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color_text, 1, cv2.LINE_AA)
    cv2.putText(img_deepRN, 'Estimate = {}'.format(center_deepRN), (430, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color_text_deepRN, 1, cv2.LINE_AA)    
    
    img_final = cv2.hconcat([img_excuse, img_deepRN])
    
    video.write(img_final)
    # cv2.imwrite('test.png', img_final)    
    
    ret, frame = benchmark.read()
    number_frame += 1
    
video.release()